In [1]:
import numpy as np
import os
import time

In [2]:
import keras

In [3]:
keras.__version__

'2.4.3'

In [4]:
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [5]:
#!pip install -q -U tensorflow-addons==0.11.2

### Utilities

In [6]:
from utils import verifyDir
from utils.networks import normalize, unnormalize, plot_data

### Dataset

In [7]:
from utils.CIFAR10 import load_real_samples

### Discriminator & Generator

In [8]:
from utils.CIFAR10 import define_discriminator
from utils.CIFAR10 import define_generator

### Semi-Supervised GAN

In [9]:
from utils.networks import define_gan

### Selecting sub-set 

In [10]:
from utils.networks import select_supervised_samples, generate_real_samples
from utils.networks import generate_fake_samples, generate_latent_points

### Training

In [11]:
# train the generator and discriminator
def train(generator_model, unsupervised_model, supervised_model, gan_model, dataset, dataset_test, 
          latent_dim=100, n_epochs=20, n_batch=100, percent_samples=1.0):
    
    # select supervised dataset
    X_sup, y_sup = select_supervised_samples(dataset, percent_samples=percent_samples)
    print("Sup samples:", X_sup.shape, y_sup.shape)
    
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    print('n_epochs=%d, n_batch=%d, batch/epoch=%d, steps=%d' % (n_epochs, n_batch, bat_per_epo, n_steps))
    
    # manually enumerate epochs
    f_history = open(f"{LOG_PATH}SSL_GAN.csv", "w")
    f_history.write("step,generator_loss,unsupervised_real_loss,unsupervised_fake_loss,supervised_loss,supervised_acc,train_loss,test_loss,train_acc,test_acc\n")
    
    #for epoch in n_epochs:
    #    for batch in range(bat_per_epo):
    for step in range(1,n_steps+1):
#         t_start = time.time()
        # update supervised discriminator (c)
        [Xsup_real, ysup_real], _ = generate_real_samples([X_sup, y_sup], n_batch)
        c_loss, c_acc = supervised_model.train_on_batch(Xsup_real, ysup_real)
        
        # update unsupervised discriminator (d)
        [X_real, _], y_real = generate_real_samples(dataset, n_batch)
        d_loss1, real_acc = unsupervised_model.train_on_batch(X_real, y_real)
        
        X_fake, y_fake = generate_fake_samples(generator_model, latent_dim, n_batch)
        d_loss2, fake_acc = unsupervised_model.train_on_batch(X_fake, y_fake)
        
        # update generator (g)
        X_gan, y_gan = generate_latent_points(latent_dim, n_batch), np.ones((n_batch, 1))
        g_loss = gan_model.train_on_batch(X_gan, y_gan)
#         t_total = (time.time() - t_start)
        # summarize loss on this batch
    
        # Train - Test
        X_train, y_train = dataset
        loss_train, acc_train = supervised_model.evaluate(X_train, y_train, verbose=0)

        # evaluate the test classifier model
        X_test, y_test = dataset_test
        loss_test, acc_test = supervised_model.evaluate(X_test, y_test, verbose=0)
        
        # Log
        print('epoch: %d | step: %d | Train: G_Loss: %.3f, ' \
              'D_unsup_loss_real: %.3f, D_unsup_acc_real:  %.2f, ' \
              'D_unsup_loss_fake: %.3f, D_unsup_acc_fake: %.2f, ' \
              'D_sup_loss: %.3f, D_sup_acc: %.2f ' \
              'Train acc: %.3f Test acc: %.3f ' %(int(step/bat_per_epo), step, g_loss,
                                                d_loss1, real_acc*100,
                                                d_loss2, fake_acc*100,
                                                c_loss, c_acc*100,
                                                 acc_train*100, acc_test*100))#, end = '\r')
        f_history.write(f"{step},{g_loss},{d_loss1},{d_loss2},{c_loss},{c_acc*100},{loss_train},{loss_test},{acc_train*100},{acc_test*100}\n")
        
        if step==1:
            plot_data(unnormalize(X_test).astype(int), 0, "test", grid_size = [10, 10], OUT_PATH=LOG_PATH)
        # evaluate the model performance every so often
        if (step) % (100) == 0 or step == 1:
            #summarize_performance(step, generator_model, supervised_model, latent_dim, dataset, dataset_test)
            # prepare fake examples
            X_generated, _ = generate_fake_samples(generator_model, latent_dim, n_samples=100)
            # scale from [-1,1] to [0,255]
            plot_data(unnormalize(X_generated).astype(int), step, "generated", grid_size = [10, 10], OUT_PATH=LOG_PATH)
            
            X_train, y_train = dataset
            _, acc = supervised_model.evaluate(X_train, y_train, verbose=0)
            print('Train Classifier Accuracy: %.3f%%\n' % (acc * 100))
            
            # evaluate the test classifier model
            X_test, y_test = dataset_test
            _, acc = supervised_model.evaluate(X_test, y_test, verbose=0)
            print('Test Classifier Accuracy: %.3f%%\n' % (acc * 100))
            # save the generator model
            filename2 = f'{LOG_PATH}generator_model_{step}.h5'
            generator_model.save(filename2)
            # save the classifier model
            filename3 = f'{LOG_PATH}supervised_model_{step}.h5'
            supervised_model.save(filename3)
            print('>Saving models Generator: %s and Supervised: %s' % (filename2, filename3))
    
    f_history.close()

### Parameters

In [12]:
input_shape = (32, 32, 3)
num_classes = 10

learning_rate =  0.0002

labeled_rate = 1.0
labeled_samples = 50000*labeled_rate

In [13]:
LOG_PATH = f"Logs/SSGAN_CIFAR10/Classifier_{int(labeled_samples)}/"
verifyDir(LOG_PATH)

### Creating models

In [14]:
# create the discriminator models
unsupervised_model, supervised_model = define_discriminator(in_shape=input_shape, n_classes=num_classes, learning_rate = learning_rate)
# create the generator
generator_model = define_generator(latent_dim=100)

In [15]:
supervised_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 128)         147584    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 128)         0     

In [16]:
unsupervised_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 128)         147584    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 128)         0   

In [17]:
generator_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              413696    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 4096)              0         
_________________________________________________________________
reshape (Reshape)            (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 8, 8, 128)         524416    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 16, 16, 128)       2622

In [18]:
# create the gan
gan_model = define_gan(generator_model, unsupervised_model, learning_rate = learning_rate)

In [19]:
gan_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              413696    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 4096)              0         
_________________________________________________________________
reshape (Reshape)            (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 8, 8, 128)         524416    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 16, 16, 128)       2622

### Loading Dataset

In [20]:
# load image data
dataset, dataset_test = load_real_samples()

(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)


### Training

In [ ]:
train(generator_model, unsupervised_model, supervised_model, gan_model, 
      dataset, dataset_test, latent_dim=100, 
      n_epochs=300, n_batch=128, percent_samples=labeled_rate)

Sup samples: (50000, 32, 32, 3) (50000,)
n_epochs=300, n_batch=128, batch/epoch=390, steps=117000
epoch: 0 | step: 1 | Train: G_Loss: 0.095, D_unsup_loss_real: 0.094, D_unsup_acc_real:  100.00, D_unsup_loss_fake: 2.400, D_unsup_acc_fake: 0.00, D_sup_loss: 2.312, D_sup_acc: 7.81 Train acc: 10.798 Test acc: 10.570 
Train Classifier Accuracy: 10.798%

Test Classifier Accuracy: 10.570%

>Saving models Generator: Logs/SSGAN_CIFAR10/Classifier_50000/generator_model_1.h5 and Supervised: Logs/SSGAN_CIFAR10/Classifier_50000/supervised_model_1.h5
epoch: 0 | step: 2 | Train: G_Loss: 0.095, D_unsup_loss_real: 0.091, D_unsup_acc_real:  100.00, D_unsup_loss_fake: 2.399, D_unsup_acc_fake: 0.00, D_sup_loss: 2.301, D_sup_acc: 10.54 Train acc: 13.200 Test acc: 13.400 
epoch: 0 | step: 3 | Train: G_Loss: 0.096, D_unsup_loss_real: 0.088, D_unsup_acc_real:  100.00, D_unsup_loss_fake: 2.398, D_unsup_acc_fake: 0.00, D_sup_loss: 2.294, D_sup_acc: 13.42 Train acc: 14.500 Test acc: 14.190 
epoch: 0 | step: 4 | 

epoch: 0 | step: 37 | Train: G_Loss: 8.009, D_unsup_loss_real: 0.418, D_unsup_acc_real:  85.16, D_unsup_loss_fake: 0.003, D_unsup_acc_fake: 100.00, D_sup_loss: 2.034, D_sup_acc: 28.24 Train acc: 26.538 Test acc: 27.190 
epoch: 0 | step: 38 | Train: G_Loss: 9.048, D_unsup_loss_real: 0.431, D_unsup_acc_real:  85.94, D_unsup_loss_fake: 0.487, D_unsup_acc_fake: 75.78, D_sup_loss: 2.010, D_sup_acc: 27.09 Train acc: 27.200 Test acc: 27.850 
epoch: 0 | step: 39 | Train: G_Loss: 7.645, D_unsup_loss_real: 0.488, D_unsup_acc_real:  82.03, D_unsup_loss_fake: 0.004, D_unsup_acc_fake: 100.00, D_sup_loss: 1.994, D_sup_acc: 27.80 Train acc: 27.622 Test acc: 28.250 
epoch: 0 | step: 40 | Train: G_Loss: 13.886, D_unsup_loss_real: 0.402, D_unsup_acc_real:  86.72, D_unsup_loss_fake: 0.511, D_unsup_acc_fake: 73.44, D_sup_loss: 1.983, D_sup_acc: 28.21 Train acc: 27.174 Test acc: 27.610 
epoch: 0 | step: 41 | Train: G_Loss: 13.396, D_unsup_loss_real: 0.717, D_unsup_acc_real:  79.69, D_unsup_loss_fake: 0.000

epoch: 0 | step: 75 | Train: G_Loss: 2.923, D_unsup_loss_real: 0.500, D_unsup_acc_real:  75.00, D_unsup_loss_fake: 0.191, D_unsup_acc_fake: 100.00, D_sup_loss: 1.925, D_sup_acc: 31.46 Train acc: 31.562 Test acc: 32.090 
epoch: 0 | step: 76 | Train: G_Loss: 3.450, D_unsup_loss_real: 0.207, D_unsup_acc_real:  91.41, D_unsup_loss_fake: 0.198, D_unsup_acc_fake: 100.00, D_sup_loss: 1.906, D_sup_acc: 32.07 Train acc: 31.144 Test acc: 32.100 
epoch: 0 | step: 77 | Train: G_Loss: 3.616, D_unsup_loss_real: 0.380, D_unsup_acc_real:  80.47, D_unsup_loss_fake: 0.165, D_unsup_acc_fake: 100.00, D_sup_loss: 1.911, D_sup_acc: 32.00 Train acc: 31.258 Test acc: 31.970 
epoch: 0 | step: 78 | Train: G_Loss: 3.318, D_unsup_loss_real: 0.443, D_unsup_acc_real:  71.09, D_unsup_loss_fake: 0.108, D_unsup_acc_fake: 100.00, D_sup_loss: 1.902, D_sup_acc: 31.90 Train acc: 31.700 Test acc: 32.380 
epoch: 0 | step: 79 | Train: G_Loss: 3.380, D_unsup_loss_real: 0.242, D_unsup_acc_real:  92.19, D_unsup_loss_fake: 0.138

epoch: 0 | step: 112 | Train: G_Loss: 3.036, D_unsup_loss_real: 0.436, D_unsup_acc_real:  78.12, D_unsup_loss_fake: 0.195, D_unsup_acc_fake: 100.00, D_sup_loss: 1.793, D_sup_acc: 35.81 Train acc: 35.478 Test acc: 36.470 
epoch: 0 | step: 113 | Train: G_Loss: 3.287, D_unsup_loss_real: 0.210, D_unsup_acc_real:  86.72, D_unsup_loss_fake: 0.180, D_unsup_acc_fake: 100.00, D_sup_loss: 1.788, D_sup_acc: 36.41 Train acc: 36.304 Test acc: 36.850 
epoch: 0 | step: 114 | Train: G_Loss: 3.549, D_unsup_loss_real: 0.261, D_unsup_acc_real:  85.94, D_unsup_loss_fake: 0.216, D_unsup_acc_fake: 100.00, D_sup_loss: 1.785, D_sup_acc: 36.79 Train acc: 36.380 Test acc: 36.900 
epoch: 0 | step: 115 | Train: G_Loss: 3.350, D_unsup_loss_real: 0.215, D_unsup_acc_real:  89.84, D_unsup_loss_fake: 0.090, D_unsup_acc_fake: 100.00, D_sup_loss: 1.787, D_sup_acc: 36.86 Train acc: 36.570 Test acc: 37.050 
epoch: 0 | step: 116 | Train: G_Loss: 3.077, D_unsup_loss_real: 0.340, D_unsup_acc_real:  85.16, D_unsup_loss_fake: 

epoch: 0 | step: 150 | Train: G_Loss: 3.082, D_unsup_loss_real: 0.151, D_unsup_acc_real:  94.53, D_unsup_loss_fake: 0.157, D_unsup_acc_fake: 98.44, D_sup_loss: 1.721, D_sup_acc: 39.12 Train acc: 38.730 Test acc: 39.020 
epoch: 0 | step: 151 | Train: G_Loss: 3.430, D_unsup_loss_real: 0.153, D_unsup_acc_real:  95.31, D_unsup_loss_fake: 0.430, D_unsup_acc_fake: 81.25, D_sup_loss: 1.716, D_sup_acc: 39.00 Train acc: 38.216 Test acc: 38.260 
epoch: 0 | step: 152 | Train: G_Loss: 3.393, D_unsup_loss_real: 0.357, D_unsup_acc_real:  78.91, D_unsup_loss_fake: 0.283, D_unsup_acc_fake: 90.62, D_sup_loss: 1.725, D_sup_acc: 38.24 Train acc: 37.928 Test acc: 37.910 
epoch: 0 | step: 153 | Train: G_Loss: 4.004, D_unsup_loss_real: 0.196, D_unsup_acc_real:  91.41, D_unsup_loss_fake: 0.533, D_unsup_acc_fake: 75.78, D_sup_loss: 1.725, D_sup_acc: 37.93 Train acc: 38.968 Test acc: 39.060 
epoch: 0 | step: 154 | Train: G_Loss: 3.222, D_unsup_loss_real: 0.290, D_unsup_acc_real:  85.16, D_unsup_loss_fake: 0.17

epoch: 0 | step: 188 | Train: G_Loss: 2.885, D_unsup_loss_real: 0.204, D_unsup_acc_real:  91.41, D_unsup_loss_fake: 0.246, D_unsup_acc_fake: 100.00, D_sup_loss: 1.667, D_sup_acc: 41.60 Train acc: 41.304 Test acc: 42.000 
epoch: 0 | step: 189 | Train: G_Loss: 3.150, D_unsup_loss_real: 0.380, D_unsup_acc_real:  78.91, D_unsup_loss_fake: 0.258, D_unsup_acc_fake: 100.00, D_sup_loss: 1.663, D_sup_acc: 41.93 Train acc: 41.548 Test acc: 42.270 
epoch: 0 | step: 190 | Train: G_Loss: 3.220, D_unsup_loss_real: 0.508, D_unsup_acc_real:  67.97, D_unsup_loss_fake: 0.239, D_unsup_acc_fake: 100.00, D_sup_loss: 1.659, D_sup_acc: 42.30 Train acc: 41.504 Test acc: 42.100 
epoch: 0 | step: 191 | Train: G_Loss: 3.362, D_unsup_loss_real: 0.277, D_unsup_acc_real:  84.38, D_unsup_loss_fake: 0.128, D_unsup_acc_fake: 100.00, D_sup_loss: 1.665, D_sup_acc: 42.10 Train acc: 41.196 Test acc: 42.000 
epoch: 0 | step: 192 | Train: G_Loss: 3.801, D_unsup_loss_real: 0.242, D_unsup_acc_real:  89.06, D_unsup_loss_fake: 

epoch: 0 | step: 225 | Train: G_Loss: 2.501, D_unsup_loss_real: 0.305, D_unsup_acc_real:  81.25, D_unsup_loss_fake: 0.314, D_unsup_acc_fake: 98.44, D_sup_loss: 1.635, D_sup_acc: 42.25 Train acc: 41.940 Test acc: 42.800 
epoch: 0 | step: 226 | Train: G_Loss: 2.584, D_unsup_loss_real: 0.385, D_unsup_acc_real:  79.69, D_unsup_loss_fake: 0.232, D_unsup_acc_fake: 100.00, D_sup_loss: 1.637, D_sup_acc: 42.80 Train acc: 42.424 Test acc: 42.790 
epoch: 0 | step: 227 | Train: G_Loss: 2.635, D_unsup_loss_real: 0.322, D_unsup_acc_real:  79.69, D_unsup_loss_fake: 0.226, D_unsup_acc_fake: 100.00, D_sup_loss: 1.625, D_sup_acc: 42.82 Train acc: 42.454 Test acc: 42.970 
epoch: 0 | step: 228 | Train: G_Loss: 2.562, D_unsup_loss_real: 0.429, D_unsup_acc_real:  79.69, D_unsup_loss_fake: 0.170, D_unsup_acc_fake: 100.00, D_sup_loss: 1.629, D_sup_acc: 42.92 Train acc: 42.178 Test acc: 42.820 
epoch: 0 | step: 229 | Train: G_Loss: 2.411, D_unsup_loss_real: 0.442, D_unsup_acc_real:  75.00, D_unsup_loss_fake: 0

epoch: 0 | step: 263 | Train: G_Loss: 2.558, D_unsup_loss_real: 0.420, D_unsup_acc_real:  75.00, D_unsup_loss_fake: 0.189, D_unsup_acc_fake: 97.66, D_sup_loss: 1.611, D_sup_acc: 43.59 Train acc: 43.692 Test acc: 44.380 
epoch: 0 | step: 264 | Train: G_Loss: 2.099, D_unsup_loss_real: 0.481, D_unsup_acc_real:  69.53, D_unsup_loss_fake: 0.281, D_unsup_acc_fake: 96.88, D_sup_loss: 1.590, D_sup_acc: 44.39 Train acc: 42.236 Test acc: 42.860 
epoch: 0 | step: 265 | Train: G_Loss: 2.310, D_unsup_loss_real: 0.263, D_unsup_acc_real:  86.72, D_unsup_loss_fake: 0.326, D_unsup_acc_fake: 99.22, D_sup_loss: 1.608, D_sup_acc: 42.76 Train acc: 42.236 Test acc: 42.910 
epoch: 0 | step: 266 | Train: G_Loss: 2.340, D_unsup_loss_real: 0.352, D_unsup_acc_real:  80.47, D_unsup_loss_fake: 0.338, D_unsup_acc_fake: 93.75, D_sup_loss: 1.612, D_sup_acc: 42.90 Train acc: 42.986 Test acc: 43.220 
epoch: 0 | step: 267 | Train: G_Loss: 2.470, D_unsup_loss_real: 0.427, D_unsup_acc_real:  77.34, D_unsup_loss_fake: 0.37

Train Classifier Accuracy: 43.158%

Test Classifier Accuracy: 42.570%

>Saving models Generator: Logs/SSGAN_CIFAR10/Classifier_50000/generator_model_300.h5 and Supervised: Logs/SSGAN_CIFAR10/Classifier_50000/supervised_model_300.h5
epoch: 0 | step: 301 | Train: G_Loss: 3.026, D_unsup_loss_real: 0.272, D_unsup_acc_real:  86.72, D_unsup_loss_fake: 0.445, D_unsup_acc_fake: 82.03, D_sup_loss: 1.608, D_sup_acc: 42.55 Train acc: 41.002 Test acc: 40.220 
epoch: 0 | step: 302 | Train: G_Loss: 2.890, D_unsup_loss_real: 0.446, D_unsup_acc_real:  74.22, D_unsup_loss_fake: 0.316, D_unsup_acc_fake: 95.31, D_sup_loss: 1.652, D_sup_acc: 40.27 Train acc: 41.868 Test acc: 41.150 
epoch: 0 | step: 303 | Train: G_Loss: 3.103, D_unsup_loss_real: 0.347, D_unsup_acc_real:  83.59, D_unsup_loss_fake: 0.483, D_unsup_acc_fake: 78.91, D_sup_loss: 1.633, D_sup_acc: 41.19 Train acc: 42.548 Test acc: 42.010 
epoch: 0 | step: 304 | Train: G_Loss: 3.357, D_unsup_loss_real: 0.237, D_unsup_acc_real:  89.84, D_unsup_los

epoch: 0 | step: 338 | Train: G_Loss: 1.785, D_unsup_loss_real: 0.615, D_unsup_acc_real:  64.06, D_unsup_loss_fake: 0.499, D_unsup_acc_fake: 81.25, D_sup_loss: 1.558, D_sup_acc: 44.30 Train acc: 45.100 Test acc: 44.680 
epoch: 0 | step: 339 | Train: G_Loss: 1.880, D_unsup_loss_real: 0.452, D_unsup_acc_real:  72.66, D_unsup_loss_fake: 0.459, D_unsup_acc_fake: 84.38, D_sup_loss: 1.546, D_sup_acc: 44.75 Train acc: 44.742 Test acc: 44.270 
epoch: 0 | step: 340 | Train: G_Loss: 1.876, D_unsup_loss_real: 0.585, D_unsup_acc_real:  60.16, D_unsup_loss_fake: 0.391, D_unsup_acc_fake: 89.06, D_sup_loss: 1.549, D_sup_acc: 44.20 Train acc: 44.802 Test acc: 44.300 
epoch: 0 | step: 341 | Train: G_Loss: 1.849, D_unsup_loss_real: 0.433, D_unsup_acc_real:  76.56, D_unsup_loss_fake: 0.490, D_unsup_acc_fake: 83.59, D_sup_loss: 1.546, D_sup_acc: 44.41 Train acc: 44.592 Test acc: 44.520 
epoch: 0 | step: 342 | Train: G_Loss: 2.009, D_unsup_loss_real: 0.352, D_unsup_acc_real:  82.81, D_unsup_loss_fake: 0.49

### Testing

In [ ]:
dataset, dataset_test = load_real_samples()

In [ ]:
X_train, y_train = dataset
_, acc = supervised_model.evaluate(X_train, y_train, verbose=0)
print('Train Classifier Accuracy: %.3f%%\n' % (acc * 100))

In [ ]:
X_test, y_test = dataset_test
_, acc = supervised_model.evaluate(X_test, y_test, verbose=0)
print('Test Classifier Accuracy: %.3f%%\n' % (acc * 100))

### Plotting

In [ ]:
import pandas as pd

In [ ]:
results_file = pd.read_csv(f"{LOG_PATH}SSL_GAN.csv")

In [ ]:
log_file = results_file.iloc[:,1:]
log_file

In [ ]:
log_file.iloc[:, [0,1,2,3]].plot(figsize=(12,8))

In [ ]:
log_file.iloc[:, [5,6]].plot(figsize=(12,8))

In [ ]:
log_file.iloc[:, [7,8]].plot(figsize=(12,8), ylim=(0,100), yticks=range(0,110,10))